# 인공 신경망
입력값 X에 가중치(W)를 곱하고 편향(b)을 더한 뒤 활성화 함수(Sigmoid, ReLU 등)을 거쳐 결과값 y를 만들어내는 것이 인공 신경망의 기본이다.  
원하는 y값을 만들어내기 위해 W와 b의 값을 변경해가면서 적절한 값을 찾아내는 최적화 과정을 학습(learning) 또는 훈련(training)  

활성화 함수(activation function)는 인공신경망을 통과한 값을 최종적으로 어떤 값으로 만들지 결정한다. 

## 간단한 분류 모델 구현
패턴을 파악해 여러 종류로 구분하는 작업을 **분류(classification)** 이라고 한다.

털과 날개가 있느냐를 기준으로 포유류와 조류를 구분하는 신경망 모델을 만들어본다.  

먼저 텐서플로와 행렬조작과 연산에 필수라 할 수 있는 NumPY 라이브러리를 임포트한다.

In [1]:
import tensorflow as tf
import numpy as np

학습에 사용할 데이터를 정의하고, 각 개체가 실제 어떤 종류인지 나타내는 레이블(분류값) 데이터를 원-핫 인코딩(one-hot encoding) 형태로 구성한다.

In [2]:
x_data = np.array(
    [[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])

y_data = np.array([
    [1, 0, 0],  # 기타
    [0, 1, 0],  # 포유류
    [0, 0, 1],  # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

`X`와 `Y`에 실측값(ground truth)을 넣어서 학습시킬 것이기 때문에 `X`와 `Y`는 플레이스홀더로 설정한다.  
그 다음 신경망을 결정하는 가중치(`W`)와 편향 변수(`b`)를 설정한다.

In [3]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

가중치(`W`)는 `[입력층(특징 수), 출력층(레이블 수)]`의 구성이고, 편향 변수(`b`)는 레이블 수인 3개의 요소를 가진 변수로 설정한다.  
이 가중치를 곱하고 편향을 더한 결과를 활성화 함수인 **ReLU**에 적용하면 신경망 구성은 끝

In [4]:
W = tf.Variable(tf.random_uniform([2, 3], -1., 1.))
b = tf.Variable(tf.zeros([3]))

In [5]:
L = tf.add(tf.matmul(X, W), b)
L = tf.nn.relu(L)

신경망을 통해 나온 출력값을 `softmax`함수를 이용해서 사용하기 쉽게 다듬어준다.  
`softmax`함수는 배열 내의 결과값들을 전체 합이 1이 되도록 만들어준다. 

In [6]:
model = tf.nn.softmax(L)

In [7]:
model

<tf.Tensor 'Softmax:0' shape=(?, 3) dtype=float32>

이제 손실함수를 작성한다.  
원-핫 인코딩 형식을 이용하는 대부분의 모델에서 사용하는 **교차 엔트로피(Cross-Entropy)** 함수를 사용한다.  
(교차 엔트로피 값은 예측값과 실제값 사이의 확률 분포 차이를 계산한 값)

> Y는 실측값이고, model은 신경망을 통해 나온 예측값이다.   
> model 값에 log를 취한 후 Y와 곱하고..  
> 행별로 값을 다 더한 후 배열 안 값의 평균을 낸다.

In [8]:
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))

이제 학습을 시켜본다.  
기본적인 경상하강법으로 최적화

In [9]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

In [10]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [11]:
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
        

10 1.0939702
20 1.0927329
30 1.0915931
40 1.0903736
50 1.0892487
60 1.0880402
70 1.0869329
80 1.0857421
90 1.0846379
100 1.0835295


예측값인 model을 바로 출력하면 `[0.2 0.7 0.1]` 같이 확률로 나오기 때문에..  
이 요소 중 가장 큰 값의 인덱스를 찾아주는 `argmax` 함수를 사용해서 레이블 값을 출력하게 한다. 

In [12]:
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)

print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

예측값: [0 0 0 0 0 2]
실제값: [0 1 2 0 0 2]


전체 학습 데이터에 대한 예측값과 실제값을 `tf.equal` 함수로 비교한 뒤 `true`, `false`로 나온 결과를 다시 `tf.cast` 함수를 이용해서 0과 1로 바꾸어 평균을 내면 간단히 정확도를 구할 수 있다.

In [13]:
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

정확도: 66.67


## 심층 신경망 구현하기
다층 신경망을 만드는 것은 매우 간단하다.  
앞서 만든 신경망 모델에 가중치와 편향을 추가하기만 하면 된다.  

입력층과 출력층은 각각 특징과 분류의 개수로 맞추고, 중간의 연결 부분(은닉층, hidden layer)은 맞닿은 층의 뉴런 수와 같도록 맞춘다.  

```
W1 = [2, 10]  -> [특징, 은닉층의 뉴런 수]
W2 = [10, 3]  -> [은닉층의 뉴런 수, 분류 수]

b1 = [10]     -> 은닉층의 뉴런 수
b2 = [3]      -> 분류 수
```

In [14]:
W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
W2 = tf.Variable(tf.random_uniform([10, 3], -1., 1.))

b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))

특정 입력값에 첫번째 가중치와 편향, 그리고 활성화 함수를 적용

In [15]:
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)

출력층을 만들기 위해 두번째 가중치와 편향을 적용하여 최종 모델을 만든다

In [16]:
model = tf.add(tf.matmul(L1, W2), b2)

손실함수와 최적화 함수를 통해 학습을 시작한다

In [19]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

이번에는 `AdamOptimizer` 최적화 함수를 사용한다. 전에 사용한 `GradientDecentOptimizer` 보다 보편적으로 성능이 좋다고 알려져있다.

In [20]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

In [22]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [23]:
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

10 0.94237685
20 0.7045836
30 0.5452649
40 0.4343115
50 0.35064474
60 0.28560346
70 0.22535618
80 0.16653451
90 0.12177116
100 0.088799536


In [24]:
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)

print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]


In [27]:
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도 %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

정확도 100.00


# 딥러닝의 세계에 온 걸 환영한다.